<a href="https://colab.research.google.com/github/plaban1981/NLP_Question_Answer_Model/blob/main/Question_generator_using_T5_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Download SQUAD dataset
* Preprocess the data
* Save as .csv file

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install datasets

     |████████████████████████████████| 290 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 51.9 MB/s 
     |████████████████████████████████| 56 kB 5.5 MB/s 
     |████████████████████████████████| 125 kB 69.1 MB/s 
     |████████████████████████████████| 1.3 MB 66.5 MB/s 
     |████████████████████████████████| 160 kB 72.1 MB/s 
     |████████████████████████████████| 271 kB 71.7 MB/s 


In [ ]:
!mkdir "/content/drive/MyDrive/t5"
!mkdir "/content/drive/MyDrive/t5/dataset"
!mkdir "/content/drive/MyDrive/t5/model"
!mkdir "/content/drive/MyDrive/t5/tokenizer"
!ls

drive  sample_data


In [ ]:
!ls "/content/drive/MyDrive/t5"

dataset  model	tokenizer


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('squad',split="train")
valid_dataset = load_dataset('squad',split="validation")

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


## View Sample SQUAD dataset

In [ ]:
from pprint import pprint
sample_valid_dataset = next(iter(valid_dataset))
pprint(sample_valid_dataset)

{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which the game would have been known as '
            '"Super

In [ ]:
sample_train_dataset = next(iter(train_dataset))
pprint(sample_train_dataset)

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

In [ ]:
context  = sample_train_dataset['context']
context

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [ ]:
question  = sample_train_dataset['question']
question

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [ ]:
sample_train_dataset['answers']

{'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}

In [ ]:
answer = sample_train_dataset['answers']['text'][0]
answer

'Saint Bernadette Soubirous'

In [ ]:
print(f"context : {context}")
print(f"question: {question}")
print(f"answer  : {answer}")

context : Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
answer  : Saint Bernadette Soubirous


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth',-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
#create empty dataframe
df_train = pd.DataFrame(columns=['context','answer','question'])
df_val = pd.DataFrame(columns=['context','answer','question'])
print(df_val)
print(df_train)

Empty DataFrame
Columns: [context, answer, question]
Index: []
Empty DataFrame
Columns: [context, answer, question]
Index: []


In [ ]:
count_long = 0
count_short = 0

In [ ]:
# iterate through training dataset and extract context
for index,val in enumerate(tqdm(train_dataset)):
  passage = val['question']
  question = val['question']
  answer = val['answers']['text'][0]
  no_of_words = len(answer.split())
  if no_of_words >= 7:
    count_long += 1
  else:
    df_train.loc[count_short] = [passage] + [answer] + [question]
    count_short += 1

print(f"count long train dataset : {count_long}")
print(f"count short train dataset : {count_short}")

  0%|          | 0/87599 [00:00<?, ?it/s]

count long train dataset : 8935
count short train dataset : 78664


In [ ]:
count_long = 0
count_short = 0
# iterate through validation dataset and extract context
for index,val in enumerate(tqdm(valid_dataset)):
  passage = val['question']
  question = val['question']
  answer = val['answers']['text'][0]
  no_of_words = len(answer.split())
  if no_of_words >= 7:
    count_long += 1
  else:
    df_val.loc[count_short] = [passage] + [answer] + [question]
    count_short += 1

print(f"count long validation dataset : {count_long}")
print(f"count short validation dataset : {count_short}")

  0%|          | 0/10570 [00:00<?, ?it/s]

count long validation dataset : 918
count short validation dataset : 9652


## Shuffle training and validation dataset

In [ ]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_val = shuffle(df_val)

In [ ]:
print(df_train.shape)
print(df_val.shape)

(78664, 3)
(9652, 3)


In [ ]:
df_train.head()

,context,answer,question
19075,Who comprises the upper house of the Marshall Islands Parliament?,twelve tribal chiefs,Who comprises the upper house of the Marshall Islands Parliament?
28794,Which part of the body relesases estrogen in females?,ovaries,Which part of the body relesases estrogen in females?
50462,Who was the mayor that proclaimed independence from the German Empire for Alsace-Lorraine?,Jacques Peirotes,Who was the mayor that proclaimed independence from the German Empire for Alsace-Lorraine?
5411,How many national airports does Portugal have?,four,How many national airports does Portugal have?
72118,What two countries divvied up the Middle East's countries after WWI?,Britain and France,What two countries divvied up the Middle East's countries after WWI?


In [ ]:
df_val.head()

,context,answer,question
336,How many years has Thomas Davis played in the NFL?,11,How many years has Thomas Davis played in the NFL?
5314,"Besides the V&A's, whose collections are under the responsibility of conservators at the V&A?",V&A Museum of Childhood,"Besides the V&A's, whose collections are under the responsibility of conservators at the V&A?"
5534,A strike by what entity resulted in a halt to production for network programs in the 2007-2008 season?,Writers Guild of America,A strike by what entity resulted in a halt to production for network programs in the 2007-2008 season?
8287,"If q=9 and a=1,2,4,5,7, or 8, how many primes would be in a progression?",infinitely many prime numbers,"If q=9 and a=1,2,4,5,7, or 8, how many primes would be in a progression?"
7454,"What is the name of the museum and research center for Near Eastern studies, that is owned by the university?",the Oriental Institute,"What is the name of the museum and research center for Near Eastern studies, that is owned by the university?"


In [ ]:
train_path = "/content/drive/MyDrive/t5/squad_t5_train.csv"
val_path = "/content/drive/MyDrive/t5/squad_t5_val.csv"
df_train.to_csv(train_path,index=False)
df_val.to_csv(val_path,index=False)

## Train T5 transformer Model

#### Install all necessary libraries

In [28]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 43.7 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 895 kB 61.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [29]:
! pip install pytorch-lightning

     |████████████████████████████████| 925 kB 5.0 MB/s 
     |████████████████████████████████| 829 kB 38.0 MB/s 
     |████████████████████████████████| 282 kB 64.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=f05a30fb0e8d35981f3f9381224f6478dcb834d7d8efc19ddf5a8f16efde6fd8
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [5]:
!pip --quiet install sentencepiece

* --quiet flag is used so that the output is not verbose

In [15]:
!pip install tokenizer

## Kill the current process and restart the runtime

In [ ]:
import os
def restart_runtime():
  os.kill(os.getpid(),9)
restart_runtime()

## 

In [6]:
train_path = "/content/drive/MyDrive/t5/squad_t5_train.csv"
val_path = "/content/drive/MyDrive/t5/squad_t5_val.csv"

In [7]:
# Check if GPU is enabled and size
!nvidia-smi

Thu Oct 21 15:32:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## import necessary packages

In [9]:
import json,glob,time,logging,random,os,argparse,re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
import pytorch_lightning as pl
from termcolor import colored
import textwrap

In [10]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [12]:
#global seeding in order for the experiments to be reproducable
pl.seed_everything(42)

Global seed set to 42


42

##Download T5 Tokenizer

In [13]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

## Exploring Tokenizer

In [20]:
from pprint import pprint
import tokenizer
sample_encoding = t5_tokenizer.encode_plus("My name is Plaban Nayak",
                                        max_length =64,
                                        pad_to_max_length = True,
                                        truncation = True,
                                        return_tensors = 'pt')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [21]:
print (sample_encoding.keys())
pprint (sample_encoding)

dict_keys(['input_ids', 'attention_mask'])
{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[  499,   564,    19,  8422,  3478,  1823, 20856,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])}


In [22]:
print (sample_encoding['input_ids'].shape)
print (sample_encoding['input_ids'].squeeze().shape)
print (sample_encoding['input_ids'])

torch.Size([1, 64])
torch.Size([64])
tensor([[  499,   564,    19,  8422,  3478,  1823, 20856,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])


* input_ids are the tokens.
* attention mask 1 where there are tokens and 0s where there are no tokens

* if the length of the tokens here is < 64 rest of the tokens is padded with 0s

* torch.Size([1, 64]) 1 batch with 64 vectors
* .squeeze - remove dimension of batch
* T5 uses sentencepiece tokenizer